# Description
After running K fold to select those data which are predicted into a wrong category, we implemented mannual recorrection. This script is to apply those correction to the original files. 

In [11]:
import pandas as pd
import numpy as np

In [12]:
model_list = ['CutDebt','IDClassifier','IfKnowDebtor','Installment','ConfirmLoan','WillingToPay']
df_others = pd.read_csv('../../MLModel/data/others/irrelevant_response_training_set.csv')
df_other_matrix = pd.read_csv('../../MLModel/data/others/strategy_mat.csv')
num2text = df_other_matrix.set_index('label').category.drop_duplicates().to_dict()

df_all_original = {}
for each in model_list:
    df_all_original[each] = pd.read_csv('../../MLModel/data/{}/mock_up_data1.csv'.format(each))

In [13]:
model = 'WillingToPay'
df_correction = pd.read_csv('../../MLModel/data/{}/labelNeedsCorrections.csv'.format(model))
other_label = max(df_all_original[model].label)
non_other_label = set(df_all_original[model].label.values) - set([other_label])
df_all_original[model] = df_all_original[model][df_all_original[model].label != other_label]
print('{} -- {}'.format(model,len(df_all_original[model])))
print('length of other: {}'.format(len(df_others)))
append_other_text = []
append_other_category = []

append_function_text = {}
append_function_category = {}

original_remove_index = {}

count_case1 = 0
count_case2 = 0
count_case3 = 0
count_case4 = 0
for index in range(len(df_correction)):
    ############ get data #####################
    cur_data = df_correction.iloc[index]
    data_from = cur_data['from']
    label = cur_data['label']
    original_text = cur_data['original_text']
    new_label = cur_data['new_label']
    ########### apply to original files #######
    if model in ['CutDebt','Installment'] and new_label == 110:
        count_case4 += 1
        if label in non_other_label:
            ### insert into others
            append_other_text.append(original_text)
            append_other_category.append(num2text[new_label])
            ### remove from original
            rindex = df_all_original[data_from][(df_all_original[data_from].label==label) &(df_all_original[data_from].split_text==original_text)].index.values
            if original_remove_index.get(data_from) is None:
                original_remove_index[data_from] = []
            original_remove_index[data_from].extend(list(rindex))
        else:
            ### update others
            rindex = df_others[df_others['文本']==original_text].index.values
            if len(rindex) == 0:
                ### insert into others
                append_other_text.append(original_text)
                append_other_category.append(num2text[new_label])
            else:    
                df_others.loc[rindex,'类别'] = num2text[110]
        continue
    if label in non_other_label and new_label >= 100:
        count_case1 += 1
        ### insert into others
        append_other_text.append(original_text)
        append_other_category.append(num2text[new_label])
        
        ### remove from original
        rindex = df_all_original[data_from][(df_all_original[data_from].label==label) &(df_all_original[data_from].split_text==original_text)].index.values
        if original_remove_index.get(data_from) is None:
            original_remove_index[data_from] = []
        original_remove_index[data_from].extend(list(rindex))
        
        
    elif label in non_other_label and new_label in non_other_label and label!= new_label:
        count_case2 += 1
        # update original data
        indexes = df_all_original[data_from][(df_all_original[data_from].label==label) &(df_all_original[data_from].split_text==original_text)].index.values
        df_all_original[data_from].loc[indexes,'label'] = new_label

    elif label == other_label and new_label in non_other_label:
        count_case3+=1
        if append_function_text.get(model) is None:
            append_function_text[model]=[]
            append_function_category[model]=[]
        append_function_text[model].append(original_text)
        append_function_category[model].append(new_label)
        
others_append = pd.DataFrame({'文本':append_other_text,'类别':append_other_category})
function_append = pd.DataFrame({'split_text':append_function_text,'label':append_function_category})

#### apply to self and others
#### drop logic
for each_key in original_remove_index:
    indexes = original_remove_index[each_key]
    df_all_original[each_key] = df_all_original[each_key].drop(index=indexes)
    print('{} -- {}'.format(each_key,len(df_all_original[each_key])))
####end drop
df_others = pd.concat([df_others,others_append],sort=True).reset_index(drop=True)
print('length of other: {}'.format(len(df_others)))
for each_key in append_function_text:
    function_append = pd.DataFrame({'split_text':append_function_text[each_key],
                                    'label':append_function_category[each_key]})
    
    df_all_original[each_key] = pd.concat([df_all_original[each_key],function_append],sort=True).reset_index(drop=True)
    print('{} -- {}'.format(each_key,len(df_all_original[each_key])))

print('case 1 has data : {}'.format(count_case1))
print('case 2 has data : {}'.format(count_case2))
print('case 3 has data : {}'.format(count_case3))
print('case 4 has data : {}'.format(count_case4))
print('total corrected is: {}'.format(count_case1+count_case2+count_case3+count_case4))
print('total file length is: {}'.format(len(df_correction)))

WillingToPay -- 6278
length of other: 4695
WillingToPay -- 5494
length of other: 5426
WillingToPay -- 5564
case 1 has data : 731
case 2 has data : 860
case 3 has data : 70
case 4 has data : 0
total corrected is: 1661
total file length is: 2733
